In [55]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import pydicom

from sklearn.model_selection import train_test_split

In [56]:
from codes.utils import get_mask, bounded, min_max_normalize, hu_clip

In [57]:
import matplotlib.pyplot as plt

# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image, 'gray')
    plt.show()

In [58]:
root = os.path.join(".", "L1_pelvic_processed", "reg_pelvic_l1")

exps = ["F222836422", "P200366339", "P221436710"]

In [59]:
cases = os.listdir(root)
cases.sort()
cases = [x.split("_")[0] for x in cases]

In [60]:
cases = [x for x in cases if x not in exps]
cases = list(set(cases))
len(cases)

46

In [61]:
train, test = train_test_split(cases, test_size=0.2)

In [62]:
train, train_id = train_test_split(train, test_size=0.5)

In [63]:
len(train), len(train_id), len(test)

(18, 18, 10)

In [66]:
import shutil
for file in train_id:
      
    ct = os.path.join(root, "{}_ct".format(file))
    dst = os.path.join(".", "L1_pelvic_processed", "train_id", "{}_ct".format(file))  
    shutil.copytree(ct, dst)
    
    reg = os.path.join(root, "{}_reg".format(file))
    dst = os.path.join(".", "L1_pelvic_processed", "train_id", "{}_reg".format(file)) 
    shutil.copytree(reg, dst)

In [53]:
tmp = os.listdir(os.path.join(".", "L2_pelvic_processed", "train"))
len(tmp)
# for temp in tmp:
#     base = temp.split("_")[0]
#     if base in exps:
#         continue
#     else:
#         shutil.rmtree(os.path.join(".", "L2_pelvic_processed", "train", temp))

90

In [ ]:
index = 21
hu_range = (-512, -257)

In [ ]:
cbct_slices = read_dicom(train_cbct_path[index])
ct_slices = read_dicom(train_ct_path[index])
region = valid_slices(cbct_slices, ct_slices)

# ditch first and last 3
cbcts = cbct_slices[region[0] + 3: region[1] - 3]
cts = ct_slices[region[0] + 3: region[1] - 3]

img = cbcts[0].pixel_array.copy()
img = hu_clip(img, -256, -257, True)
img = (img * 255).astype(np.uint8)

In [ ]:
np.unique(img)

In [ ]:
plt.hist(img.flatten(), color='c')

In [ ]:
cnts, hier = cv2.findContours(img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
c = max(cnts, key = cv2.contourArea)

In [ ]:
tmp = np.stack((img, img, img), axis=-1)
cv2.drawContours(tmp, [c], 0, (0, 255, 0), 3)
plt.figure(0, figsize=(6,6))
plt.imshow(tmp)

In [ ]:
mask = np.zeros((img.shape[0], img.shape[1], 3))
cv2.drawContours(mask, [c], 0, (255, 255, 255), -1)
plt.figure(0, figsize=(6,6))
plt.imshow(mask)